In [1]:
from ketos.neural_networks.densenet import DenseNetInterface
from ketos.data_handling.data_feeding import BatchGenerator
import ketos.data_handling.database_interface as dbi
import tensorflow as tf
import random
from operator import itemgetter
import gc
import numpy as np

2021-12-01 03:08:41.791402: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-01 03:08:42.248586: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10477 MB memory:  -> device: 0, name: NVIDIA TITAN V, pci bus id: 0000:a1:00.0, compute capability: 7.0
/home/sadman/anaconda3/envs/deeplearning/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [2]:
db = dbi.open_file("../../saved_database/db_test_3.0.h5", 'a')

bh_data_cb50 = dbi.open_table(db, '/CB50/BH/data')
neg_data_cb50 = dbi.open_table(db, '/CB50/Neg/data')
bh_data_cb300 = dbi.open_table(db, '/CB300/BH/data')
neg_data_cb300 = dbi.open_table(db, '/CB300/Neg/data')

In [3]:
def transform_batch(X, Y):
  x = X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
  y = tf.one_hot(Y, depth=2, axis=1).numpy()
  return x, y

def extrat_data_label_from_table(data_table_list, x_col_name='data'):
  """ Extract data/spectrogram and class labels from table's group

      Args:
          data_table_list: table
              List of tables read from hdf5 file
          x_col_name: string
              Name of column contains the data, default value 'data'
      Returns:
          x_data: list
              List of all spectrogram data/array of groups
          y_label: list
              List of all spectrogram class labels of groups       
                      
  """
  x_data, y_label = [], []
  for data_table in data_table_list:
      for index in range(data_table.nrows):
          x_data.append(data_table[index][x_col_name])
          y_label.append(data_table[index]['label'])
  return x_data, y_label

def release_list(a):
  """ Deletes list to clear up the memory 
      Args:
        a: list
            List of data or class labels
  
  """
  del a[:]
  del a

In [4]:
pos_x_data, pos_y_label = extrat_data_label_from_table([bh_data_cb50, bh_data_cb300])
neg_x_data, neg_y_label = extrat_data_label_from_table([neg_data_cb50, neg_data_cb300])
x_all, y_all = pos_x_data+neg_x_data, pos_y_label+neg_y_label

In [6]:
# Remove unused variables to clear up memory
del bh_data_cb50
del neg_data_cb50
del bh_data_cb300
del neg_data_cb300

release_list(pos_x_data)
release_list(pos_y_label)

release_list(neg_x_data)
release_list(neg_y_label)

gc.collect()

71

In [7]:
batch_size = 8
epochs = 100

# The ratio of data keep for training
train_ratio=0.7
train_indices = random.sample(range(len(y_all)), int(len(y_all)*train_ratio))

val_indices = [x for x in range(len(y_all)) if x not in train_indices]

# Get train/validation elements from list 
x_train = list(itemgetter(*train_indices)(x_all))
y_train = list(itemgetter(*train_indices)(y_all))
x_val = list(itemgetter(*val_indices)(x_all))
y_val = list(itemgetter(*val_indices)(y_all))

train_generator = BatchGenerator(batch_size=batch_size, x=np.asarray(x_train), y=np.asarray(y_train), 
                                output_transform_func=transform_batch,
                                shuffle=True, refresh_on_epoch_end=True)

val_generator = BatchGenerator(batch_size=batch_size, x=np.asarray(x_train), y=np.asarray(y_train), 
                                output_transform_func=transform_batch,
                                shuffle=True, refresh_on_epoch_end=True)



In [8]:
#Define the parameters for the early-stopping monitor
monitor = {"metric":"val_loss",                    
          "decreasing": True,
          "period":10,
          "min_epochs":epochs//3,
          "delta" : 0.01,
          "baseline":0.0 }

detector_version='bh_detector_v02'
densenet = DenseNetInterface.build_from_recipe_file("densenet_recipe_1.json")
densenet.checkpoint_dir = "checkpoints_"+detector_version
densenet.log_dir = "log_"+detector_version
densenet.train_generator = train_generator
densenet.val_generator = val_generator
densenet.early_stopping_monitor = monitor

densenet.train_loop(n_epochs=epochs, log_csv=True, validate=True, early_stopping=True)

2021-12-01 03:09:27.402841: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-01 03:09:29.173941: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201



Epoch: 1 
train_loss: 0.6019188761711121
train_BinaryAccuracy: 0.690 train_Precision: 0.614 train_Recall: 0.584 
val_loss: 1.8711897134780884
val_BinaryAccuracy: 0.396 val_Precision: 0.396 val_Recall: 0.999 


Focus metric val_loss

Epoch: 2 
train_loss: 0.47943150997161865
train_BinaryAccuracy: 0.775 train_Precision: 0.712 train_Recall: 0.723 
val_loss: 0.44138237833976746
val_BinaryAccuracy: 0.818 val_Precision: 0.733 val_Recall: 0.847 


Focus metric val_loss

Epoch: 3 
train_loss: 0.41355207562446594
train_BinaryAccuracy: 0.821 train_Precision: 0.775 train_Recall: 0.772 
val_loss: 0.3359050750732422
val_BinaryAccuracy: 0.865 val_Precision: 0.848 val_Recall: 0.802 


Focus metric val_loss

Epoch: 4 
train_loss: 0.3488689064979553
train_BinaryAccuracy: 0.861 train_Precision: 0.820 train_Recall: 0.831 
val_loss: 1.3630236387252808
val_BinaryAccuracy: 0.721 val_Precision: 0.588 val_Recall: 0.983 


Focus metric val_loss

Epoch: 5 
train_loss: 0.3134872615337372
train_BinaryAccuracy: 0

val_loss: 0.3624820411205292
val_BinaryAccuracy: 0.927 val_Precision: 0.849 val_Recall: 0.991 


Focus metric val_loss

Epoch: 22 
train_loss: 0.14096631109714508
train_BinaryAccuracy: 0.948 train_Precision: 0.921 train_Recall: 0.951 
val_loss: 0.19969186186790466
val_BinaryAccuracy: 0.938 val_Precision: 0.873 val_Recall: 0.988 


Focus metric val_loss

Epoch: 23 
train_loss: 0.1383078247308731
train_BinaryAccuracy: 0.949 train_Precision: 0.921 train_Recall: 0.952 
val_loss: 0.12157038599252701
val_BinaryAccuracy: 0.959 val_Precision: 0.918 val_Recall: 0.984 


Focus metric val_loss

Epoch: 24 
train_loss: 0.1322755068540573
train_BinaryAccuracy: 0.951 train_Precision: 0.924 train_Recall: 0.954 
val_loss: 0.11703165620565414
val_BinaryAccuracy: 0.957 val_Precision: 0.943 val_Recall: 0.949 


Focus metric val_loss

Epoch: 25 
train_loss: 0.12041766941547394
train_BinaryAccuracy: 0.957 train_Precision: 0.932 train_Recall: 0.961 
val_loss: 0.1300048828125
val_BinaryAccuracy: 0.954 val_Pre


Focus metric val_loss

Epochs without improvement: 3

Current value: tf.Tensor(0.069103256, shape=(), dtype=float32)

Best value: tf.Tensor(0.0503958, shape=(), dtype=float32)

Epoch: 40 
train_loss: 0.08106062561273575
train_BinaryAccuracy: 0.970 train_Precision: 0.955 train_Recall: 0.970 
val_loss: 0.06411202996969223
val_BinaryAccuracy: 0.977 val_Precision: 0.972 val_Recall: 0.969 


Focus metric val_loss

Epochs without improvement: 4

Current value: tf.Tensor(0.06411203, shape=(), dtype=float32)

Best value: tf.Tensor(0.0503958, shape=(), dtype=float32)

Epoch: 41 
train_loss: 0.08113589882850647
train_BinaryAccuracy: 0.969 train_Precision: 0.953 train_Recall: 0.970 
val_loss: 0.04790572449564934
val_BinaryAccuracy: 0.982 val_Precision: 0.962 val_Recall: 0.994 


Focus metric val_loss

Epochs without improvement: 5

Current value: tf.Tensor(0.047905724, shape=(), dtype=float32)

Best value: tf.Tensor(0.0503958, shape=(), dtype=float32)

Epoch: 42 
train_loss: 0.06250539422035217


val_loss: 0.042826954275369644
val_BinaryAccuracy: 0.985 val_Precision: 0.965 val_Recall: 0.998 


Focus metric val_loss

Epochs without improvement: 1

Current value: tf.Tensor(0.042826954, shape=(), dtype=float32)

Best value: tf.Tensor(0.023260279, shape=(), dtype=float32)

Epoch: 55 
train_loss: 0.03594142943620682
train_BinaryAccuracy: 0.988 train_Precision: 0.982 train_Recall: 0.987 
val_loss: 0.025188542902469635
val_BinaryAccuracy: 0.993 val_Precision: 0.993 val_Recall: 0.989 


Focus metric val_loss

Epochs without improvement: 2

Current value: tf.Tensor(0.025188543, shape=(), dtype=float32)

Best value: tf.Tensor(0.023260279, shape=(), dtype=float32)

Epoch: 56 
train_loss: 0.0441097617149353
train_BinaryAccuracy: 0.984 train_Precision: 0.976 train_Recall: 0.983 
val_loss: 0.0197865292429924
val_BinaryAccuracy: 0.994 val_Precision: 0.994 val_Recall: 0.990 


Focus metric val_loss

Epochs without improvement: 3

Current value: tf.Tensor(0.01978653, shape=(), dtype=float32)

B

{'checkpoint_name': 'cp-0053.ckpt'}

In [9]:
densenet.save_model("/home/sadman/arctic_mammal/trained_models/"+detector_version+".kt", audio_repr_file="/home/sadman/arctic_mammal/code/prepare_db/spec_config.json")